# Investigate cause of getGainFromFlatPairs error.
Craig Lage - 24Jul22

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## First, look at the impact of the Lupton calculation on a pure Poisson distribution

In [ ]:
gain = 2.0
level = 10000
sizex = 1000
sizey = 1000
f1 = np.random.poisson(level, size=(sizex, sizey)) / gain
f2 = np.random.poisson(level, size=(sizex, sizey)) / gain
invGainUsual = np.mean((f1 - f2) * (f1 - f2)) / np.mean(f1 + f2)
gainUsual = 1.0 / invGainUsual
invGainLupton = np.mean((f1 - f2) * (f1 - f2) / (f1 + f2))
gainLupton = 1.0 / invGainLupton
print(f"Usual gain = {gainUsual}, Lupton gain = {gainLupton}")

## So the gain is well calculated, and the use of <(I1-I2)^2 / (I1+I2)> vs <(I1-I2)^2>/<(I1+I2)> makes no difference.

## Now do the same calculation with a 20% flux gradient

In [ ]:
gradient = 0.2
f1 = np.zeros([sizex, sizey])
f2 = np.zeros([sizex, sizey])
for i in range(sizey):
    this_level = level + level * gradient / sizey * i
    f1[:,i] = np.random.poisson(this_level, size=sizex) / gain
    f2[:,i] = np.random.poisson(this_level, size=sizex) / gain
invGainUsual = np.mean((f1 - f2) * (f1 - f2)) / np.mean(f1 + f2)
gainUsual = 1.0 / invGainUsual
invGainLupton = np.mean((f1 - f2) * (f1 - f2) / (f1 + f2))
gainLupton = 1.0 / invGainLupton
print(f"Usual gain = {gainUsual}, Lupton gain = {gainLupton}")

## The flux gradient makes no difference.  The gain is still well calculated.

## Now try introducing read noise to see how well the correction works
## The calculations below are copied from the cp_pipe code.

In [ ]:
for readNoise in [2.0, 10.0, 20.0]:
    f1 = np.random.poisson(level, size=(sizex, sizey)) / gain
    f2 = np.random.poisson(level, size=(sizex, sizey)) / gain
    f1Noise = np.random.normal(0.0, readNoise, size=(sizex, sizey))
    f1 += f1Noise
    f2Noise = np.random.normal(0.0, readNoise, size=(sizex, sizey))
    f2 += f2Noise
    const = np.mean((f1 - f2) * (f1 - f2) / (f1 + f2))
    mu = np.mean((f1 + f2) / 2.0)
    for correctionType in ['NONE', 'SIMPLE', 'FULL']:
        gainLupton = 1. / const
        if correctionType == 'SIMPLE':
            gainLupton = 1/(const - (1/mu)*(readNoise**2 - (1/2*gainLupton**2)))
        elif correctionType == 'FULL':
            root = np.sqrt(mu**2 - 2*mu*const + 2*readNoise**2)
            denom = (2*const*mu - 2*readNoise**2)
            positiveSolution = (root + mu)/denom
            gainLupton = positiveSolution
        print(f"readNoise = {readNoise} , correctionType = {correctionType}, gain = {gainLupton}")

## The correction successfully corrects for the read noise

## What if we over-estimate the read noise by a factor of two?

In [ ]:
for readNoise in [2.0, 10.0, 20.0]:
    f1 = np.random.poisson(level, size=(sizex, sizey)) / gain
    f2 = np.random.poisson(level, size=(sizex, sizey)) / gain
    f1Noise = np.random.normal(0.0, readNoise, size=(sizex, sizey))
    f1 += f1Noise
    f2Noise = np.random.normal(0.0, readNoise, size=(sizex, sizey))
    f2 += f2Noise
    const = np.mean((f1 - f2) * (f1 - f2) / (f1 + f2))
    mu = np.mean((f1 + f2) / 2.0)
    readNoise *= 2.0 # Overestimate the read noise
    for correctionType in ['NONE', 'SIMPLE', 'FULL']:
        gainLupton = 1. / const
        if correctionType == 'SIMPLE':
            gainLupton = 1/(const - (1/mu)*(readNoise**2 - (1/2*gainLupton**2)))
        elif correctionType == 'FULL':
            root = np.sqrt(mu**2 - 2*mu*const + 2*readNoise**2)
            denom = (2*const*mu - 2*readNoise**2)
            positiveSolution = (root + mu)/denom
            gainLupton = positiveSolution
        print(f"readNoise = {readNoise/2.0} , correctionType = {correctionType}, gain = {gainLupton}")

## So an incorrect estimate of the read noise can account for the gain being wrong.